In [ ]:
# 📦 Step 1: Install Required Packages
# !pip install opencv-python mediapipe pyautogui --quiet

# 📥 Step 2: Import Libraries
import cv2
import mediapipe as mp
import pyautogui
import math

# 🧠 Helper Function to Detect Which Fingers Are Up
def fingers_up(landmarks):
    tips_ids = [4, 8, 12, 16, 20]
    fingers = []

    # Thumb
    fingers.append(1 if landmarks[4].x < landmarks[3].x else 0)

    # Other four fingers
    for id in range(1, 5):
        fingers.append(1 if landmarks[tips_ids[id]].y < landmarks[tips_ids[id] - 2].y else 0)

    return fingers

# 🖱️ Step 3: Initialize Camera and Mediapipe
cap = cv2.VideoCapture(0)
hand_detector = mp.solutions.hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
drawing_utils = mp.solutions.drawing_utils
screen_width, screen_height = pyautogui.size()

clicking = False
scrolling = False
right_clicking = False
zoom_mode = False
zoom_start_dist = None
tab_opened = False
desktop_minimized = False
photo_taken = False
photo_counter = 0

# 🌀 Step 4: Main Loop for Gesture Control
while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    output = hand_detector.process(rgb_frame)
    hands = output.multi_hand_landmarks

    if hands:
        for hand in hands:
            drawing_utils.draw_landmarks(frame, hand, mp.solutions.hands.HAND_CONNECTIONS)
            landmarks = hand.landmark

            # Get finger landmarks
            index_finger = landmarks[8]
            thumb_finger = landmarks[4]
            middle_finger = landmarks[12]
            ring_finger = landmarks[16]

            # Move mouse with index finger
            x = int(index_finger.x * screen_width)
            y = int(index_finger.y * screen_height)
            pyautogui.moveTo(x, y)

            # Finger coordinates for distance calculation
            ix, iy = int(index_finger.x * frame.shape[1]), int(index_finger.y * frame.shape[0])
            tx, ty = int(thumb_finger.x * frame.shape[1]), int(thumb_finger.y * frame.shape[0])
            mx, my = int(middle_finger.x * frame.shape[1]), int(middle_finger.y * frame.shape[0])
            rx, ry = int(ring_finger.x * frame.shape[1]), int(ring_finger.y * frame.shape[0])

            click_dist = math.hypot(tx - ix, ty - iy)
            scroll_dist = math.hypot(tx - mx, ty - my)
            right_click_dist = math.hypot(rx - tx, ry - ty)

            # ✨ Left Click: Thumb + Index
            if click_dist < 30 and not clicking:
                clicking = True
                pyautogui.click()
            elif click_dist >= 30:
                clicking = False

            # 🔄 Scroll: Thumb + Middle
            if scroll_dist < 30 and not scrolling:
                scrolling = True
                pyautogui.scroll(-100)
            elif scroll_dist >= 30:
                scrolling = False

            # 🖱️ Right Click: Thumb + Ring
            if right_click_dist < 30 and not right_clicking:
                right_clicking = True
                pyautogui.click(button='right')
            elif right_click_dist >= 30:
                right_clicking = False

            # 🔍 Zoom (Experimental): Thumb + Index Distance Change
            current_zoom_dist = math.hypot(tx - ix, ty - iy)
            if zoom_mode and zoom_start_dist is not None:
                zoom_diff = current_zoom_dist - zoom_start_dist
                if abs(zoom_diff) > 10:
                    pyautogui.keyDown('ctrl')
                    pyautogui.scroll(int(zoom_diff * 2))
                    pyautogui.keyUp('ctrl')
                    zoom_start_dist = current_zoom_dist
            else:
                zoom_start_dist = current_zoom_dist
                zoom_mode = True

            # 🧠 Detect Finger Pattern
            fingers = fingers_up(landmarks)

            # ✌️ Open new tab in Chrome: Index + Middle only
            if fingers[1] == 1 and fingers[2] == 1 and fingers[3] == 0 and fingers[4] == 0:
                if not tab_opened:
                    pyautogui.hotkey('ctrl', 't')
                    tab_opened = True
            else:
                tab_opened = False

            # 👍 or 👎 Thumb Only: Minimize or Restore Desktop
            if fingers[0] == 1 and fingers[1:] == [0, 0, 0, 0]:
                if landmarks[4].y < landmarks[3].y:  # Thumb up
                    if not desktop_minimized:
                        pyautogui.hotkey('win', 'd')  # Minimize all
                        desktop_minimized = True
                elif landmarks[4].y > landmarks[3].y:  # Thumb down
                    if desktop_minimized:
                        pyautogui.hotkey('win', 'd')  # Restore all
                        desktop_minimized = False

            # 📸 Take Photo: Thumb + Pinky Only (🤙)
            if fingers[0] == 1 and fingers[1] == 0 and fingers[2] == 0 and fingers[3] == 0 and fingers[4] == 1:
                if not photo_taken:
                    photo_filename = f"photo_{photo_counter}.png"
                    cv2.imwrite(photo_filename, frame)
                    print(f"📸 Photo taken: {photo_filename}")
                    photo_taken = True
                    photo_counter += 1
            else:
                photo_taken = False

    # 🖥️ Show Camera Feed
    cv2.imshow("🖐️ Virtual Mouse (Gesture Controlled)", frame)

    # Exit on ESC key
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()


📸 Photo taken: photo_0.png
📸 Photo taken: photo_1.png
📸 Photo taken: photo_2.png
